In [2]:
# Importing Python libraries
import pickle
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2
import skimage.morphology as morp
from skimage.filters import rank
from sklearn.utils import shuffle
import csv
import os
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from sklearn.metrics import confusion_matrix
import os

# is it using the GPU?
print(tf.test.gpu_device_name())

# Show current TensorFlow version
tf.__version__


from matplotlib import pyplot as plt
import cv2
from keras.preprocessing import image
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from collections import defaultdict, namedtuple

Using TensorFlow backend.


In [3]:
## unzip the 
import zipfile
from keras.models import load_model
import pandas as pd


qe1 = "RAIN"
qe2 = "DARKNESS"
qe3 = "MOTIONBLUR"
qe4 = "BACKLIGHTSUN"

model_file_paths = ["TSD/h_files/deepCNN{}.h5".format(qe1), 
                    "TSD/h_files/deepCNN{}.h5".format(qe2), 
                    "TSD/h_files/deepCNN{}.h5".format(qe3), 
                    "TSD/h_files/deepCNN{}.h5".format(qe4)
                   ]

quality_effect = "LARGER_PROTOTYPE/"
zip_path =  "TSD/LARGER_PROTOTYPE/"

In [11]:
#df = pd.read_csv(zip_path + "/{}/{}ALLcombined.csv".format(quality_effect,quality_effect))
zip_path = "/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/GTSRB_augmentations/Augmented/output_data/RANDOM_IMAGES/COMPILED_TESTSET/"
df = pd.read_csv(zip_path + "/ALL_combined.csv")
print(df.head())
print(df.columns)

df = df.sample(frac=1)

print(len(df))

   BACKLIGHTSUN  DARKNESS  MOTIONBLUR  RAIN  SignType      Unnamed: 0  \
0           0.0      25.0        25.0   0.0        30    1557943525_0   
1           0.0      25.0        25.0   0.0        26    1557943525_1   
2           0.0      25.0        25.0   0.0        10   1557943525_10   
3           0.0      25.0        25.0   0.0        22  1557943525_100   
4           0.0      25.0        25.0   0.0        10  1557943525_101   

                 class       detailed_class  \
0  MOTIONBLUR_DARKNESS  MOTIONBLUR_DARKNESS   
1  MOTIONBLUR_DARKNESS  MOTIONBLUR_DARKNESS   
2  MOTIONBLUR_DARKNESS  MOTIONBLUR_DARKNESS   
3  MOTIONBLUR_DARKNESS  MOTIONBLUR_DARKNESS   
4  MOTIONBLUR_DARKNESS  MOTIONBLUR_DARKNESS   

                                       full_img_pth       hour  rain_sensor  \
0    1557943525_0_UC2_MOTIONBLUR-25_DARKNESS-25.png  20.108238     1.318375   
1    1557943525_1_UC2_MOTIONBLUR-25_DARKNESS-25.png  22.524323     0.885584   
2   1557943525_10_UC2_MOTIONBLUR-25_DARKN

In [12]:
def gray_scale(image):
    """
    Convert images to gray scale.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)


def image_normalize(image):
    """
    Normalize images to [0, 1] scale.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    image = np.divide(image, 255)
    return image

def local_histo_equalize(image):
    """
    Apply local histogram equalization to grayscale images.
        Parameters:
            image: A grayscale image.
    """
    kernel = morp.disk(30)
    img_local = rank.equalize(image, selem=kernel)
    return img_local


def preprocess(data):
    """
    Applying the preprocessing steps to the input data.
        Parameters:
            data: An np.array compatible with plt.imshow.
    """
    gray_images = list(map(gray_scale, data))
    equalized_images = list(map(local_histo_equalize, gray_images))
    n_training = data.shape
    normalized_images = np.zeros((n_training[0], n_training[1], n_training[2]))
    for i, img in enumerate(equalized_images):
        normalized_images[i] = image_normalize(img)
    normalized_images = normalized_images[..., None]
    return normalized_images

In [13]:
from keras.preprocessing import image

def create_image_tensor(IMG_PTH):
    img = image.load_img(IMG_PTH, target_size=(32, 32))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0) 
    img_tensor /= 255.  
    new_image = img_tensor.reshape(img_tensor.shape[0], 3, 32, 32)
    return new_image


def create_predictions(model, image):
    DEEP_CNN = list()  
    model_input = create_image_tensor(image)
    quality_model_prediction = model.predict(model_input)
    prediction_label = quality_model_prediction.argmax()
    DEEP_CNN.append(prediction_label)
    
    prediction_by_class = model.predict_proba(model_input) 
    prediction_prob = np.amax((prediction_by_class))
    DEEP_CNN.append(prediction_prob)
    return DEEP_CNN

def evaluate_deep_CNN_images(input_model, new_test_images, deep=False):
    if deep: 
        new_test_images = new_test_images.reshape(new_test_images.shape[0], 1, 32, 32)
    y_prob = input_model.predict(new_test_images)
    
    y_classes = y_prob.argmax(axis=-1)
    print(y_classes)
    return y_classes,y_prob

In [6]:
RAIN_MODEL = load_model(model_file_paths[0])
DARKNESS_MODEL = load_model(model_file_paths[1])
MOTIONBLUR_MODEL = load_model(model_file_paths[2])
BACKLIGHTSUN_MODEL = load_model(model_file_paths[3])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [14]:
base_path  = zip_path

def process_data(base_path, data_frame):
    print("Resizing images & processing")
    in_data = []
    cr_labels = []
    for row, column in data_frame.iterrows():
        img_id = column['full_img_pth']
        directory = column['class']
        img_pth = base_path + directory + "/" + img_id
        if os.path.isfile(img_pth):
            img = cv2.imread(img_pth)
            img = cv2.resize(img, (32,32))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            in_data.append(img)
            cr_labels.append(column['SignType'])
        else: 
            print("IMAGE PATH NOT FOUND:{}".format(img_pth))
    return in_data, cr_labels
## creates test images
input_data, correct_labels = process_data(base_path, df)

Resizing images & processing


In [15]:
new_test_images = preprocess(np.asarray(input_data))

In [18]:
print(len(new_test_images))

31400


## Updates the dataframe with prediction classes and probabilities

In [19]:

darkness_classes, darkness_prob = evaluate_deep_CNN_images(DARKNESS_MODEL,new_test_images, True)
rain_classes, rain_prob = evaluate_deep_CNN_images(RAIN_MODEL,new_test_images, True)
mb_classes, mb_prob = evaluate_deep_CNN_images(MOTIONBLUR_MODEL,new_test_images, True)
backlight_classes, backlight_prob = evaluate_deep_CNN_images(BACKLIGHTSUN_MODEL,new_test_images, True)

## adds new rows to the subset of the testing images 
df['DARKNESS_classes'] = darkness_classes
df['RAIN_classes'] = rain_classes
df['MOTIONBLUR_classes'] = mb_classes
df['BACKLIGHTSUN_classes'] = backlight_classes


[0 0 0 ... 0 0 0]
[1 1 0 ... 1 0 2]
[0 0 0 ... 0 1 0]
[0 0 0 ... 0 0 0]


In [22]:
save_path = "TSD/DT_PROTOTYPE/data.csv"
df.to_csv(save_path)